## `dask.array`

<img src="https://github.com/ualberta-rcg/python-dask/blob/master/notebooks/assets/array.png?raw=1" width="25%" align="right">
Dask array provides a parallel, larger-than-memory, n-dimensional array using blocked algorithms. Simply put: distributed Numpy.

*  **Parallel**: Uses all of the cores on your computer
*  **Larger-than-memory**:  Lets you work on datasets that are larger than your available memory by breaking up your array into many small pieces, operating on those pieces in an order that minimizes the memory footprint of your computation, and effectively streaming data from disk.
*  **Blocked Algorithms**:  Perform large computations by performing many smaller computations

**Related Documentation**

* [Documentation](http://dask.readthedocs.io/en/latest/array.html)
* [API reference](http://dask.readthedocs.io/en/latest/array-api.html)

## Blocked Algorithms

A *blocked algorithm* executes on a large dataset by breaking it up into many small blocks.

For example, consider taking the sum of a billion numbers.  We might instead break up the array into 1,000 chunks, each of size 1,000,000, take the sum of each chunk, and then take the sum of the intermediate sums.

We achieve the intended result (one sum on one billion numbers) by performing many smaller results (one thousand sums on one million numbers each, followed by another sum of a thousand numbers.)

We do exactly this with Python and NumPy in the following example.

## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [ ]:
# NOTE: COLAB, don't run this!
# (or add the additional argument: diagnostics_port=None)
# Home users: only run this if you want a dashboard

from dask.distributed import Client, progress
client = Client(processes=False, threads_per_worker=2,
                n_workers=1, memory_limit='2GB')
client

## Create Random array

This creates a 10000x10000 array of random numbers, represented as many numpy arrays of size 1000x1000 (or smaller if the array cannot be divided evenly). In this case there are 100 (10x10) numpy arrays of size 1000x1000.

In [1]:
import dask.array as da
x = da.random.random((10000, 10000), chunks=(1000, 1000))
x

dask.array<random_sample, shape=(10000, 10000), dtype=float64, chunksize=(1000, 1000), chunktype=numpy.ndarray>

The following operations resemble operations from NumPy.

In [2]:
# y is the sum of x with it's transpose
y = x + x.T
y

dask.array<add, shape=(10000, 10000), dtype=float64, chunksize=(1000, 1000), chunktype=numpy.ndarray>

In [3]:
# z is the mean along the first axis after slicing y
# Along the first axis we take every second column
# We take the last half of the rows (5000 of them)
z = y[::2, 5000:]
z

dask.array<getitem, shape=(5000, 5000), dtype=float64, chunksize=(500, 1000), chunktype=numpy.ndarray>

In [4]:
# v is the mean of z along the first axis (5000 numbers)
v = z.mean(axis=1)
v

dask.array<mean_agg-aggregate, shape=(5000,), dtype=float64, chunksize=(500,), chunktype=numpy.ndarray>

So far nothing has been calculated, but the task graph has been assembled.

This entire sequence of operations could be replaced by the one line:
```python
v = (x + x.T)[::2, 5000:].mean(axis=1)
```

We can inspect the task graph for this operation:

In [5]:
v.visualize()

We can now compute the result:

In [6]:
%%time
result = v.compute()

CPU times: user 2.51 s, sys: 448 ms, total: 2.96 s
Wall time: 2.49 s


In [7]:
print("Length of results:", len(result))
print("First 5 means:", result[:5])

Length of results: 5000
First 5 means: [1.00753095 0.99076469 1.00496829 1.00276385 1.00569749]


In [8]:
client.close()

NameError: name 'client' is not defined

## Try the computation again with different scheduling options

### Distributed: one worker, four threads-per-worker

In [9]:
# NOTE: For Colab add the following additional
#   argument: diagnostics_port=None

from dask.distributed import Client, progress
client = Client(processes=False, threads_per_worker=4,
                n_workers=1, memory_limit='2GB', diagnostics_port=None)
client

INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at: inproc://172.28.0.12/283/1
INFO:distributed.scheduler:  dashboard at:  http://172.28.0.12:8787/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.worker:      Start worker at: inproc://172.28.0.12/283/4
INFO:distributed.worker:         Listening to:          inproc172.28.0.12
INFO:distributed.worker:          Worker name:                          0
INFO:distributed.worker:         dashboard at:          172.28.0.12:35721
INFO:distributed.worker:Waiting to connect to: inproc://172.28.0.12/283/1
INFO:distributed.worker:-------------------------------------------------
INFO:distributed.worker:              Threads:                          4
INFO:distributed.worker:               Memory:                   1.86 GiB
INFO:distrib

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://172.28.0.12:8787/status,
Dashboard: http://172.28.0.12:8787/status,Workers: 1
Total threads: 4,Total memory: 1.86 GiB
Status: running,Using processes: False
Comm: inproc://172.28.0.12/283/1,Workers: 1
Dashboard: http://172.28.0.12:8787/status,Total threads: 4
Started: Just now,Total memory: 1.86 GiB
Comm: inproc://172.28.0.12/283/4,Total threads: 4
Dashboard: http://172.28.0.12:35721/status,Memory: 1.86 GiB
Nanny: None,


In [10]:
%%time
result = v.compute()

CPU times: user 2.06 s, sys: 321 ms, total: 2.38 s
Wall time: 2.27 s


In [11]:
client.close()

INFO:distributed.scheduler:Remove client Client-f6e2dd75-0d97-11f0-811b-0242ac1c000c
INFO:distributed.core:Received 'close-stream' from inproc://172.28.0.12/283/6; closing.
INFO:distributed.scheduler:Remove client Client-f6e2dd75-0d97-11f0-811b-0242ac1c000c
INFO:distributed.scheduler:Close client connection: Client-f6e2dd75-0d97-11f0-811b-0242ac1c000c
INFO:distributed.scheduler:Retire worker addresses (stimulus_id='retire-workers-1743360691.262281') [0]
INFO:distributed.worker:Stopping worker at inproc://172.28.0.12/283/4. Reason: worker-close
INFO:distributed.worker:Removing Worker plugin shuffle
INFO:distributed.core:Received 'close-stream' from inproc://172.28.0.12/283/5; closing.
INFO:distributed.scheduler:Remove worker addr: inproc://172.28.0.12/283/4 name: 0 (stimulus_id='handle-worker-cleanup-1743360691.271079')
INFO:distributed.scheduler:Lost all workers
INFO:distributed.core:Connection to inproc://172.28.0.12/283/1 has been closed.
INFO:distributed.scheduler:Closing scheduler.

### Distributed: four workers, one thread-per-worker

In [12]:
# NOTE: For Colab add the following additional
#   argument: diagnostics_port=None
from dask.distributed import Client, progress
client = Client(processes=False, threads_per_worker=1,
                n_workers=4, memory_limit='2GB', diagnostics_port=None)
client

INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at: inproc://172.28.0.12/283/10
INFO:distributed.scheduler:  dashboard at:  http://172.28.0.12:8787/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.worker:      Start worker at: inproc://172.28.0.12/283/13
INFO:distributed.worker:         Listening to:          inproc172.28.0.12
INFO:distributed.worker:          Worker name:                          0
INFO:distributed.worker:         dashboard at:          172.28.0.12:42009
INFO:distributed.worker:Waiting to connect to: inproc://172.28.0.12/283/10
INFO:distributed.worker:-------------------------------------------------
INFO:distributed.worker:              Threads:                          1
INFO:distributed.worker:               Memory:                   1.86 GiB
INFO:distributed.worker:      Local Directory: /tmp/dask-scratch-space/worker-r4jvo4yv
INFO:distributed.worker:-------------------------------------------------

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://172.28.0.12:8787/status,
Dashboard: http://172.28.0.12:8787/status,Workers: 4
Total threads: 4,Total memory: 7.45 GiB
Status: running,Using processes: False
Comm: inproc://172.28.0.12/283/10,Workers: 4
Dashboard: http://172.28.0.12:8787/status,Total threads: 4
Started: Just now,Total memory: 7.45 GiB
Comm: inproc://172.28.0.12/283/13,Total threads: 1
Dashboard: http://172.28.0.12:42009/status,Memory: 1.86 GiB
Nanny: None,


In [13]:
%%time
result = v.compute()

CPU times: user 2.28 s, sys: 334 ms, total: 2.62 s
Wall time: 1.53 s


In [14]:
client.close()

INFO:distributed.scheduler:Remove client Client-0352c8cc-0d98-11f0-811b-0242ac1c000c
INFO:distributed.core:Received 'close-stream' from inproc://172.28.0.12/283/21; closing.
INFO:distributed.scheduler:Remove client Client-0352c8cc-0d98-11f0-811b-0242ac1c000c
INFO:distributed.scheduler:Close client connection: Client-0352c8cc-0d98-11f0-811b-0242ac1c000c
INFO:distributed.scheduler:Retire worker addresses (stimulus_id='retire-workers-1743360706.5311983') [0, 1, 2, 3]
INFO:distributed.worker:Stopping worker at inproc://172.28.0.12/283/13. Reason: worker-close
INFO:distributed.worker:Stopping worker at inproc://172.28.0.12/283/15. Reason: worker-close
INFO:distributed.worker:Stopping worker at inproc://172.28.0.12/283/17. Reason: worker-close
INFO:distributed.worker:Stopping worker at inproc://172.28.0.12/283/19. Reason: worker-close
INFO:distributed.worker:Removing Worker plugin shuffle
INFO:distributed.worker:Removing Worker plugin shuffle
INFO:distributed.worker:Removing Worker plugin sh

### Distributed: one worker, one thread-per-worker

This is basically serial.

In [15]:
# NOTE: For Colab add the following additional
#   argument: diagnostics_port=None
from dask.distributed import Client, progress
client = Client(processes=False, threads_per_worker=1,
                n_workers=1, memory_limit='2GB', diagnostics_port=None)
client

INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at: inproc://172.28.0.12/283/43
INFO:distributed.scheduler:  dashboard at:  http://172.28.0.12:8787/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.worker:      Start worker at: inproc://172.28.0.12/283/46
INFO:distributed.worker:         Listening to:          inproc172.28.0.12
INFO:distributed.worker:          Worker name:                          0
INFO:distributed.worker:         dashboard at:          172.28.0.12:36483
INFO:distributed.worker:Waiting to connect to: inproc://172.28.0.12/283/43
INFO:distributed.worker:-------------------------------------------------
INFO:distributed.worker:              Threads:                          1
INFO:distributed.worker:               Memory:                   1.86 GiB
INFO:distributed.worker:      Local Directory: /tmp/dask-scratch-space/worker-4wgwinhy
INFO:distributed.worker:-------------------------------------------------

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://172.28.0.12:8787/status,
Dashboard: http://172.28.0.12:8787/status,Workers: 1
Total threads: 1,Total memory: 1.86 GiB
Status: running,Using processes: False
Comm: inproc://172.28.0.12/283/43,Workers: 1
Dashboard: http://172.28.0.12:8787/status,Total threads: 1
Started: Just now,Total memory: 1.86 GiB
Comm: inproc://172.28.0.12/283/46,Total threads: 1
Dashboard: http://172.28.0.12:36483/status,Memory: 1.86 GiB
Nanny: None,


In [16]:
%%time
result = v.compute()

CPU times: user 1.6 s, sys: 271 ms, total: 1.87 s
Wall time: 1.84 s


In [17]:
client.close()

INFO:distributed.scheduler:Remove client Client-0b919a69-0d98-11f0-811b-0242ac1c000c
INFO:distributed.core:Received 'close-stream' from inproc://172.28.0.12/283/48; closing.
INFO:distributed.scheduler:Remove client Client-0b919a69-0d98-11f0-811b-0242ac1c000c
INFO:distributed.scheduler:Close client connection: Client-0b919a69-0d98-11f0-811b-0242ac1c000c
INFO:distributed.scheduler:Retire worker addresses (stimulus_id='retire-workers-1743360718.491223') [0]
INFO:distributed.worker:Stopping worker at inproc://172.28.0.12/283/46. Reason: worker-close
INFO:distributed.worker:Removing Worker plugin shuffle
INFO:distributed.core:Received 'close-stream' from inproc://172.28.0.12/283/47; closing.
INFO:distributed.scheduler:Remove worker addr: inproc://172.28.0.12/283/46 name: 0 (stimulus_id='handle-worker-cleanup-1743360718.499518')
INFO:distributed.scheduler:Lost all workers
INFO:distributed.core:Connection to inproc://172.28.0.12/283/43 has been closed.
INFO:distributed.scheduler:Closing sched

### Non-distributed: synchronous scheduler

In [18]:
%%time
result = v.compute(scheduler='synchronous')

CPU times: user 1.02 s, sys: 323 ms, total: 1.35 s
Wall time: 1.34 s


### Non-distributed: threaded scheduler

In [19]:
%%time
result = v.compute(scheduler='threads')

CPU times: user 1.57 s, sys: 429 ms, total: 2 s
Wall time: 1.09 s


### Non-distributed: multiple processes

In [20]:
%%time
result = v.compute(scheduler='processes')

CPU times: user 1.08 s, sys: 1.93 s, total: 3.01 s
Wall time: 8.92 s


[On to the next notebook (`dask.dataframe`)](05-dataframe.ipynb) ...